# Module 3: Creating a Simple CNN

## Practice: Training a Convolutional Neural Network on CIFAR-10

In this notebook, we will train a simple Convolutional Neural Network (CNN) on the CIFAR-10 dataset. CIFAR-10 consists of 60,000 32x32 color images across 10 classes (airplane, car, bird, cat, etc.).

Imagine an IoT device (e.g., Raspberry Pi + camera) capturing images and classifying them either on the device (edge computing) or in the cloud. This example demonstrates how a CNN learns to recognize objects in images.


## 1. Imports and Dataset Loading

We start by importing necessary libraries and loading the CIFAR-10 dataset directly from Keras. We'll also normalize pixel values to the [0,1] range.


In this cell we import the necessary libraries and load the CIFAR-10 dataset:

- **tensorflow**: For building and training deep learning models.
- **datasets, layers, models** from Keras: To handle data and define architectures.
- **matplotlib.pyplot** and **numpy**: For plotting and numerical operations.

We load CIFAR-10 (60,000 32×32 color images in 10 classes), normalize pixel values by dividing by 255, and print the shapes of the training and test sets.

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from keras.layers import Input
import matplotlib.pyplot as plt
import numpy as np
import requests
from io import BytesIO
from PIL import Image

print("Libraries imported succesfully")

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()

print("Dataset loaded successfully")

# Normalize pixel values
x_train = x_train.astype('float32') / 255.0
x_test  = x_test.astype('float32') / 255.0

## 2. Visualizing the Data

Let's look at a few samples from the training set to understand what the images look like. We'll display the first 5 images with their labels.


In this cell we define class names and display the first 5 training images:

- Create a `class_names` list with human-readable labels for each class.
- Use `plt.subplots` to create a row of 5 subplots.
- For each image, display it, set the title to its label, and hide the axes.

This helps inspect example data and understand class distribution.

In [ ]:
class_names = ["airplane","automobile","bird","cat","deer",
               "dog","frog","horse","ship","truck"]

# Show first 5 images
fig, axes = plt.subplots(1, 5, figsize=(5, 2))
for i in range(5):
    axes[i].imshow(x_train[i])
    label = y_train[i][0]  # because y_train[i] is an array
    axes[i].set_title(class_names[label])
    axes[i].axis('off')
plt.show()

## 3. Building the CNN Model

We will construct a simple CNN with two convolutional blocks followed by pooling layers, then flatten the output and add fully connected layers for classification.


In this cell we define a simple Convolutional Neural Network (CNN) architecture:

- Start with a Keras **Sequential** model.  
- Use an explicit **Input** layer to specify the input shape (32×32×3).  
- First convolutional block: `Conv2D` with 32 filters of size 3×3 and **ReLU** activation, followed by `MaxPooling2D` (2×2).  
- Second convolutional block: `Conv2D` with 64 filters of size 3×3 and **ReLU** activation, followed by `MaxPooling2D` (2×2).  
- Flatten the feature maps with `Flatten`.  
- Fully connected layers: a `Dense` layer with 64 neurons and **ReLU**, and a final `Dense` layer with 10 neurons and **softmax** for 10-class classification.  
- Finally, display the model summary with `model.summary()`.  

In [ ]:
model = models.Sequential([
    Input(shape=(32, 32, 3)),

    # Primeiro bloque convolucional
    layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Segundo bloque convolucional
    layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),

    # Aplanar os mapas de características
    layers.Flatten(),

    # Capas densas para clasificación
    layers.Dense(64, activation='relu'),
    layers.Dense(10, activation='softmax')
])

print("Model created")

## 4. Compiling and Training the Model

We'll compile the model using the Adam optimizer and sparse categorical crossentropy loss, then train for 5 epochs with a batch size of 64.


In this cell we compile and train the defined model:

- **Optimizer:** Adam.
- **Loss Function:** sparse_categorical_crossentropy (suitable for integer labels).
- **Metric:** accuracy.
- **Fit Parameters:** 5 epochs, batch size 64, using the test set for validation.

The returned `history` object records loss and accuracy over training and validation.

In [ ]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

history = model.fit(x_train, y_train, epochs=5, 
                    validation_data=(x_test, y_test), 
                    batch_size=64)

## 5. Evaluating the Model

Evaluate the trained CNN on the test set to measure its accuracy.


In this cell we evaluate the model on the test dataset:

- Call `model.evaluate` with `x_test` and `y_test`.
- Retrieve `test_loss` and `test_acc`.
- Print the final test accuracy to quantify model performance.

In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
print(f"Test accuracy: {test_acc:.2f}")

## 6. Classifying an External Image via URL

To demonstrate how our trained CNN can classify new, real-world images, we can load an image from any URL:

1. We prompt the user to enter an image URL.
2. We download the image using `requests` and open it with `PIL.Image`.
3. We convert it to RGB, resize it to the required input shape (32×32), and normalize pixel values to `[0.0, 1.0]`.
4. We expand dimensions to create a batch of size 1, which is needed for `model.predict`.
5. We run the model to obtain prediction probabilities.
6. We select the class with the highest probability and display both the class name and the probability.

In [ ]:
# Prompt for the image URL
url = input("Image URL: ")

# Download and open the image
response = requests.get(url)
img = Image.open(BytesIO(response.content)).convert("RGB")

# Resize to 32×32 and normalize pixel values to [0,1]
img = img.resize((32, 32))
img_array = np.array(img).astype('float32') / 255.0

# Create a batch of size 1
img_batch = np.expand_dims(img_array, axis=0)

# Make a prediction
preds = model.predict(img_batch)
class_idx = np.argmax(preds, axis=1)[0]
prob = preds[0][class_idx]

# Display the result
print(f"Predicted class: {class_names[class_idx]}  —  Probability: {prob:.2%}")